In [1]:
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import scipy.io as scio
import numpy as np
import os 
import pac
from numba import  cuda, jit
import time 
import pickle
from joblib import Parallel, delayed

import multiprocessing
import joblib

### Load Task Files and ERP

In [2]:
Path = '/mnt/D/Mastersharif/MasterProject/Working_with_kiani/git/parkinson-pac/task_track_files/task_track_file_matlab_windows_double_calculation.csv'
list_tasks_info = pd.read_csv(Path, usecols=[
                              'local_file_path', 'raw_data_file_name', 'Group_file_type', 'preprocessed_two', 'ERP_npy_file_with_normalization_and_baseline_correction', 'ERP_nobaseline_npy_file'])
num_task_file = list_tasks_info.shape[0]
list_tasks_info

,local_file_path,raw_data_file_name,preprocessed_two,ERP_npy_file_with_normalization_and_baseline_correction,ERP_nobaseline_npy_file,Group_file_type
0,sub-001/ses-01/eeg/,sub-001_ses-01_task-Rest_eeg.set,pre_sub-001_ses-01_task-Rest_eeg_double.set,ERP_base_correct_normalization_sub-001_ses-01_...,ERP_nb_correct_sub-001_ses-01_task-Rest_eeg_db...,PD_ON
1,sub-001/ses-02/eeg/,sub-001_ses-02_task-Rest_eeg.set,pre_sub-001_ses-02_task-Rest_eeg_double.set,ERP_base_correct_normalization_sub-001_ses-02_...,ERP_nb_correct_sub-001_ses-02_task-Rest_eeg_db...,PD_OFF
2,sub-002/ses-01/eeg/,sub-002_ses-01_task-Rest_eeg.set,pre_sub-002_ses-01_task-Rest_eeg_double.set,ERP_base_correct_normalization_sub-002_ses-01_...,ERP_nb_correct_sub-002_ses-01_task-Rest_eeg_db...,PD_ON
3,sub-002/ses-02/eeg/,sub-002_ses-02_task-Rest_eeg.set,pre_sub-002_ses-02_task-Rest_eeg_double.set,ERP_base_correct_normalization_sub-002_ses-02_...,ERP_nb_correct_sub-002_ses-02_task-Rest_eeg_db...,PD_OFF
4,sub-003/ses-01/eeg/,sub-003_ses-01_task-Rest_eeg.set,pre_sub-003_ses-01_task-Rest_eeg_double.set,ERP_base_correct_normalization_sub-003_ses-01_...,ERP_nb_correct_sub-003_ses-01_task-Rest_eeg_db...,CTL
...,...,...,...,...,...,...
70,sub-046/ses-01/eeg/,sub-046_ses-01_task-Rest_eeg.set,pre_sub-046_ses-01_task-Rest_eeg_double.set,ERP_base_correct_normalization_sub-046_ses-01_...,ERP_nb_correct_sub-046_ses-01_task-Rest_eeg_db...,CTL
71,sub-047/ses-01/eeg/,sub-047_ses-01_task-Rest_eeg.set,pre_sub-047_ses-01_task-Rest_eeg_double.set,ERP_base_correct_normalization_sub-047_ses-01_...,ERP_nb_correct_sub-047_ses-01_task-Rest_eeg_db...,CTL
72,sub-048/ses-01/eeg/,sub-048_ses-01_task-Rest_eeg.set,pre_sub-048_ses-01_task-Rest_eeg_double.set,ERP_base_correct_normalization_sub-048_ses-01_...,ERP_nb_correct_sub-048_ses-01_task-Rest_eeg_db...,CTL
73,sub-049/ses-01/eeg/,sub-049_ses-01_task-Rest_eeg.set,pre_sub-049_ses-01_task-Rest_eeg_double.set,ERP_base_correct_normalization_sub-049_ses-01_...,ERP_nb_correct_sub-049_ses-01_task-Rest_eeg_db...,CTL


In [23]:
list_tasks_info['Group_file_type'][5]

'PD_OFF'

### Load ERP

In [3]:
# Path = 'D:\Mastersharif\MasterProject\data\parkinsons-oddball'
## linux version path
Path = '/mnt/D/Mastersharif/MasterProject/data/parkinsons-oddball'
ERP_collection = np.zeros([3, 25, 3, 63, 601])
list_tasks_info.shape[0]
counter_PD_on = 0
counter_PD_off = 0
counter_CTL = 0
for i in np.arange(num_task_file):
    Path_to_save_file = os.path.join(
        Path, list_tasks_info['local_file_path'][i], list_tasks_info['ERP_npy_file_with_normalization_and_baseline_correction'][i])
    ERP_array = np.load(Path_to_save_file)
    if list_tasks_info['Group_file_type'][i] == 'PD_ON':
        ERP_collection[0, counter_PD_on, :, :, :] = ERP_array
        counter_PD_on = counter_PD_on + 1
    elif list_tasks_info['Group_file_type'][i] == 'PD_OFF':
        ERP_collection[1, counter_PD_off, :, :, :] = ERP_array
        counter_PD_off = counter_PD_off + 1
    elif list_tasks_info['Group_file_type'][i] == 'CTL':
        ERP_collection[2, counter_CTL, :, :, :] = ERP_array
        counter_CTL = counter_CTL + 1


### Check double calculation in ERP collection and indivisual calculation

In [24]:
i = 3
Path = '/mnt/D/Mastersharif/MasterProject/data/parkinsons-oddball'
Path_to_save_file = os.path.join(
        Path, list_tasks_info['local_file_path'][i], list_tasks_info['ERP_npy_file_with_normalization_and_baseline_correction'][i])
print(Path_to_save_file)
ERP_new_version=np.load(Path_to_save_file)

print(ERP_new_version.shape)
# np.any(ERP_collection[1,1,0,0,:] == ERP_new_version[0,0,:])



/mnt/D/Mastersharif/MasterProject/data/parkinsons-oddball/sub-002/ses-02/eeg/ERP_base_correct_normalization_sub-002_ses-02_task-Rest_eeg_db.npy
(3, 63, 601)


In [28]:
i=0
j=23
np.any(ERP_collection[1,1,i,j,:] != ERP_new_version[i,j,:])

False

In [4]:
## the order of channel names is compared with matlab order and there is no problem
with open('../data/channels_name.pkl', 'rb') as f:
    channels_name = pickle.load(f)

print(channels_name)    
selected_channels = ['Fz','Pz','Cz','FCz']
# selected_channels = ['FCz']
list_index = [channels_name.index(i) for i in selected_channels]

list_index

['Fp1', 'Fz', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 'CP5', 'CP1', 'Pz', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'TP10', 'CP6', 'CP2', 'Cz', 'C4', 'T8', 'FT10', 'FC6', 'FC2', 'F4', 'F8', 'Fp2', 'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'FCz', 'C1', 'C5', 'TP7', 'CP3', 'P1', 'P5', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'P6', 'P2', 'CP4', 'TP8', 'C6', 'C2', 'FC4', 'FT8', 'F6', 'F2', 'AF4', 'AF8']


[1, 12, 23, 39]

In [1]:
window_size = 200 # in ms
down_sample_factor = 1
window_sample = int(window_size*(0.5/down_sample_factor))
frequency_h_n = 76
frequency_l_n = 23
frequency_range_high = np.linspace(25,100,frequency_h_n).astype(np.int64)
frequency_range_low = np.linspace(8,30,frequency_l_n).astype(np.int64)
PAC_feature_sub_on_electrodes = np.zeros([3, 25, 3, 63,frequency_h_n ,frequency_l_n,6])
i=1
# @jit(parallel=True,fastmath=True)
def PAC_caluculation():
    for i in np.arange(3):
        for sub in np.arange(25):
            for j in np.arange(3):
                for ch in list_index:
                    for h,fre_h in enumerate(frequency_range_high):
                        for l,fre_l in enumerate(frequency_range_low):
                            for t in np.arange(6):
                                PAC_feature_sub_on_electrodes[i,sub,j,ch,h,l,t] = pac.tfMVL(ERP_collection[i,sub,j,ch,t*window_sample:(t+1)*window_sample],[fre_h,fre_h],[fre_l,fre_l],500)                          
if __name__ == "__main__":

    pool_obj = multiprocessing.Pool()
    
    PAC_caluculation()
    np.save(f'..\data\PAC_time_feature_8_30_25_100.npy',PAC_feature_sub_on_electrodes)
  

NameError: name 'np' is not defined

In [8]:

window_size = 200 # in ms
down_sample_factor = 1
window_sample = int(window_size*(0.5/down_sample_factor))
frequency_h_n = 67
frequency_l_n = 23
frequency_range_high = np.linspace(25,100,frequency_h_n).astype(np.int64)
frequency_range_low = np.linspace(8,30,frequency_l_n).astype(np.int64)


PAC_feature_sub_on_electrodes = np.memmap('..\data\Test.mmap',dtype=float ,shape=(3, 25, 3, 63,frequency_h_n ,frequency_l_n,6),mode='w+')
# PAC_feature_sub_on_electrodes = np.zeros([3, 25, 3, 63,frequency_h_n ,frequency_l_n,6])
print(PAC_feature_sub_on_electrodes.flags)
def wrapper(i,sub,j,ch,h,fre_h,l,fre_l,t):
    PAC_feature_sub_on_electrodes[i,sub,j,ch,h,l,t] =pac.tfMVL(ERP_collection[i,sub,j,ch,t*window_sample:(t+1)*window_sample],[fre_h,fre_h],[fre_l,fre_l],500)                          
    

Parallel(n_jobs=6)(delayed(wrapper)(i, sub, j, ch ,h,fre_h,l,fre_l,t) for i in range(1) for sub in range(1) for j in range(1) for ch in list_index
for h,fre_h in enumerate(frequency_range_high) for l,fre_l in enumerate(frequency_range_low) for t in np.arange(6))

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False



### Compare PAC values of mine and kiani in both mode time series and total


#### Load PAC of kiani total

In [19]:
Path_kiani = '/mnt/D/Mastersharif/MasterProject/Working_with_kiani/git/parkinson-pac/data/Check_PAC_double_calculation/sub-004_ses-01_task-Rest_mvl_2ds_1_200_double.npz'


a = np.load(Path_kiani)

a['S200'].shape

i=1

In [28]:
print(a['S201'].shape)
index = np.linspace(0,62,63).astype(int)
# print(index)
b = a['S201'][index,index,:,:]
print(b.shape)
i = 10
np.any(a['S202'][1,1,:,:] != b[1,:,:])


(63, 63, 200, 50)
(63, 200, 50)


True

In [30]:
i = 54
np.any(a['S201'][i,i,:,:] != b[i,:,:])
# print(a['S200'][0,0,:,:])

False

In [ ]:
frequency_range_high = np.linspace(1,200,200).astype(int)
frequency_range_low = np.linspace(1,50,50).astype(int)
frequency_h_n = frequency_range_high.shape[0]
frequency_l_n = frequency_range_low.shape[0]

Path_save_test = '/mnt/D/Mastersharif/MasterProject/Working_with_kiani/git/parkinson-pac/data/Check_PAC_double_calculation/PAC_total.mmap'
PAC_feature_sub_on_electrodes = np.memmap(Path_save_test,dtype=float ,shape=(3, 63,frequency_h_n ,frequency_l_n),mode='w+')

frequency_range_high = np.linspace(1,200,200).astype(int)-1
frequency_range_low = np.linspace(1,50,50).astype(int)-1

def wrapper(st,ch,h,l):
    PAC_feature_sub_on_electrodes[st,ch,h,l] = pac.tfMVL(ERP_collection[0,0,st,ch,:],[h+1,h+1],[l+1,l+1],500)                          
    

Parallel(n_jobs=8)(delayed(wrapper)(st,ch,fre_h,fre_l) for st in range(3) for ch in range(63)
for fre_h in frequency_range_high for fre_l in frequency_range_low)

In [34]:
# Path_save_test = '/mnt/D/Mastersharif/MasterProject/Working_with_kiani/git/parkinson-pac/data/Check_PAC_double_calculation/PAC_total.mmap'

# PAC_feature_sub_on_electrodes = np.memmap(Path_save_test,dtype=float ,shape=(3, 63,frequency_h_n ,frequency_l_n),mode='r+')

i,j,k = 45,110,47
print(b[i,j,k]*1e-12)

pac.tfMVL(ERP_collection[1,2,1,i,:],[j+1,j+1],[k+1,k+1],500)


0.003164027533151236


0.0031640275331512435

#### Load PAC of kiani time series

In [52]:
Path_kiani = '/mnt/D/Mastersharif/MasterProject/Working_with_kiani/git/parkinson-pac/data/Check_PAC_double_calculation/sub-004_ses-01_task-Rest_mvl_2d_times_1_200_double.npz'


a = np.load(Path_kiani)


a['S200'].shape


(63, 200, 50, 6)

In [67]:
i,j,k,t = 12,12,5,2
print(a['S200'][i,j,k,t]*1e-12)

print(pac.tfMVL(ERP_collection[1,2,0,i,100*t:100*(t+1)],[j+1,j+1],[k+1,k+1],500))



2.6239425092790274
2.6239425092790265


## Calculation for low frequency alpha



In [16]:
window_size = 200 # in ms
down_sample_factor = 1
window_sample = int(window_size*(0.5/down_sample_factor))
frequency_h_n = 15
frequency_l_n = 23
frequency_range_high = np.linspace(10,24,frequency_h_n).astype(np.int64)
frequency_range_low = np.linspace(8,30,frequency_l_n).astype(np.int64)




In [19]:
print(frequency_range_high)
print(frequency_range_low)
window_sample

[10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]
[ 8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30]


100

In [20]:
PAC_feature_sub_on_electrodes = np.memmap('../data/PAC_time_feature_8_30_10_24.mmap',dtype=float ,shape=(3, 25, 3, 63,frequency_h_n ,frequency_l_n,6),mode='w+')

def wrapper(i,sub,j,ch,h,fre_h,l,fre_l,t):
    PAC_feature_sub_on_electrodes[i,sub,j,ch,h,l,t] = pac.tfMVL(ERP_collection[i,sub,j,ch,t*window_sample:(t+1)*window_sample],[fre_h,fre_h],[fre_l,fre_l],500)                          
    

Parallel(n_jobs=8)(delayed(wrapper)(i, sub, j, ch ,h,fre_h,l,fre_l,t) for i in range(3) for sub in range(25) for j in range(3) for ch in list_index
for h,fre_h in enumerate(frequency_range_high) for l,fre_l in enumerate(frequency_range_low) for t in np.arange(6))

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [7]:
PAC_feature_sub_on_electrodes = np.memmap('..\data\PAC_time_feature_8_30_25_100_v1_total_h_fre.mmap',dtype=float ,shape=(3, 25, 3, 63,frequency_h_n ,frequency_l_n,6),mode='w+')
# PAC_feature_sub_on_electrodes = np.zeros([3, 25, 3, 63,frequency_h_n ,frequency_l_n,6])
print(PAC_feature_sub_on_electrodes.flags)
def wrapper(i,sub,j,ch,h,fre_h,l,fre_l,t):
    PAC_feature_sub_on_electrodes[i,sub,j,ch,h,l,t] = pac.tfMVL(ERP_collection[i,sub,j,ch,t*window_sample:(t+1)*window_sample],[fre_h,fre_h],[fre_l,fre_l],500)                          
    

Parallel(n_jobs=12)(delayed(wrapper)(i, sub, j, ch ,h,fre_h,l,fre_l,t) for i in range(3) for sub in range(25) for j in range(3) for ch in list_index
for h,fre_h in enumerate(frequency_range_high) for l,fre_l in enumerate(frequency_range_low) for t in np.arange(6))

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False



[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [15]:
frequency_range_high.shape[0]
frequency_range_high
frequency_range_high_1

array([ 25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,
        38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,
        51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,
        64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,
        77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,
        90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100], dtype=int64)

## Check the completion of calculation


In [25]:
frequency_l_n
frequency_h_n
# frequency_range_high
# frequency_range_low

15

In [26]:
b = np.memmap('../data/PAC_time_feature_8_30_10_24.mmap',dtype=float ,shape=(3, 25, 3, 63,frequency_h_n ,frequency_l_n,6),mode='r+')
print(b.shape)
index_sorted = np.zeros(shape=(9*25*4*frequency_l_n*frequency_h_n,6))
counter = 0
for i in np.arange(3):
    for sub in np.arange(25):
        for j in np.arange(3):
            for ch in list_index:
                for h in np.arange(frequency_range_high.shape[0]):
                    for l in np.arange(frequency_range_low.shape[0]):
                        if np.any(b[i,sub,j,ch,h,l,:]==0):
                            index_sorted[counter,:] = [i,sub,j,ch,h,l]
                        counter = counter + 1 
print(index_sorted.max())

(3, 25, 3, 63, 15, 23, 6)
0.0


## Make a Compelete array of PAC feature
### 10-100,5-30

In [32]:
frequency_h_n = 15
frequency_l_n = 23
a = np.memmap('../data/PAC_time_feature_8_30_10_24.mmap',dtype=float ,shape=(3, 25, 3, 63,frequency_h_n ,frequency_l_n,6),mode='r+')
frequency_h_n = 91
frequency_l_n = 3
b = np.memmap('../data/PAC_time_feature_5_7_10_100.mmap',dtype=float ,shape=(3, 25, 3, 63,frequency_h_n ,frequency_l_n,6),mode='r+')
frequency_h_n = 76
frequency_l_n = 23
c = np.memmap('../data/PAC_time_feature_8_30_25_100_total.mmap',dtype=float ,shape=(3, 25, 3, 63,frequency_h_n ,frequency_l_n,6),mode='r+')
frequency_h_n = 91
frequency_l_n = 26
d = np.memmap('../data/PAC_time_feature_5_30_10_100.mmap',dtype=float ,shape=(3, 25, 3, 63,frequency_h_n ,frequency_l_n,6),mode='w+')

print(d[:,:,:,:,15:,3:,:].shape)
print(c[:,:,:,:,:,:,:].shape)
print(d[:,:,:,:,:,0:3,:].shape)
print(b[:,:,:,:,:,:,:].shape)
print(d[:,:,:,:,0:15,3:,:].shape)
print(a[:,:,:,:,:,:,:].shape)


d[:,:,:,:,15:,3:,:] = c[:,:,:,:,:,:,:]
d[:,:,:,:,:,0:3,:] = b[:,:,:,:,:,:,:]
d[:,:,:,:,0:15,3:,:] = a[:,:,:,:,:,:,:]




(3, 25, 3, 63, 76, 23, 6)
(3, 25, 3, 63, 76, 23, 6)
(3, 25, 3, 63, 91, 3, 6)
(3, 25, 3, 63, 91, 3, 6)
(3, 25, 3, 63, 15, 23, 6)
(3, 25, 3, 63, 15, 23, 6)


##  Move PAC Calculation from central to forental regions

### Steps 
#### Choose channels 
#### Run parallel calculation for range 5-30 10-100
#### Save calculations

# First step select channels

In [2]:
with open('../data/channels_name.pkl', 'rb') as f:
    channels_name = pickle.load(f)
print(channels_name)    
selected_channels = ['FC3','FC4','AF3','AF4','F3','F4']
# selected_channels = ['FCz']
list_index = [channels_name.index(i) for i in selected_channels]

list_index

['Fp1', 'Fz', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 'CP5', 'CP1', 'Pz', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'TP10', 'CP6', 'CP2', 'Cz', 'C4', 'T8', 'FT10', 'FC6', 'FC2', 'F4', 'F8', 'Fp2', 'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'FCz', 'C1', 'C5', 'TP7', 'CP3', 'P1', 'P5', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'P6', 'P2', 'CP4', 'TP8', 'C6', 'C2', 'FC4', 'FT8', 'F6', 'F2', 'AF4', 'AF8']


[38, 57, 33, 61, 2, 29]

## Run parallel
### Parameter setting


In [5]:
window_size = 200 # in ms
down_sample_factor = 1
window_sample = int(window_size*(0.5/down_sample_factor))
frequency_h_n = 91
frequency_l_n = 26
frequency_range_high = np.linspace(10,100,frequency_h_n).astype(np.int64)
frequency_range_low = np.linspace(5,30,frequency_l_n).astype(np.int64)


In [14]:
window_sample
list_index

1.6703496526133792

In [15]:
PAC_feature_sub_on_electrodes = np.memmap('../data/PAC_time_feature_5_30_10_100_frontal_regions.mmap',dtype=float ,shape=(3, 25, 3, 63,frequency_h_n ,frequency_l_n,6),mode='r+')

def wrapper(i,sub,j,ch,h,fre_h,l,fre_l,t):
    if PAC_feature_sub_on_electrodes[i,sub,j,ch,h,l,t] == 0:
        PAC_feature_sub_on_electrodes[i,sub,j,ch,h,l,t] = pac.tfMVL(ERP_collection[i,sub,j,ch,t*window_sample:(t+1)*window_sample],[fre_h,fre_h],[fre_l,fre_l],500)                          
    

Parallel(n_jobs=8)(delayed(wrapper)(i, sub, j, ch ,h,fre_h,l,fre_l,t) for i in range(3) for sub in range(25) for j in range(3) for ch in list_index
for h,fre_h in enumerate(frequency_range_high) for l,fre_l in enumerate(frequency_range_low) for t in np.arange(6))

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [21]:
frequency_h_n
frequency_l_n
frequency_range_low
frequency_range_high

array([ 10,  11,  12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,
        23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,
        36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,
        49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,
        62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,
        75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,
        88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100])

In [22]:
PAC_feature_sub_on_electrodes = np.memmap('../data/PAC_time_feature_5_30_10_100_frontal_regions.mmap',dtype=float ,shape=(3, 25, 3, 63,frequency_h_n ,frequency_l_n,6),mode='r+')
counter = 0 
index_sorted = np.zeros(shape=(9*25*6*91*26,6))
for i in np.arange(3):
    for sub in np.arange(25):
        for j in np.arange(3):
            for ch in list_index:
                for h in np.arange(frequency_range_high.shape[0]):
                    for l in np.arange(frequency_range_low.shape[0]):
                        if np.any(PAC_feature_sub_on_electrodes[i,sub,j,ch,h,l,:]==0):
                            index_sorted[counter,:] = [i,sub,j,ch,h,l]
                        counter = counter + 1 
print(index_sorted.max())

0.0


In [9]:
counter = 0
for i in np.arange(3):
    for sub in np.arange(25):
        for j in np.arange(3):
            for ch in list_index:
                for h in np.arange(frequency_range_high.shape[0]):
                    for l in np.arange(frequency_range_low.shape[0]):
                        if np.any(PAC_feature_sub_on_electrodes[i,sub,j,ch,h,l,:]==0):
                            index_sorted[counter,:] = [i,sub,j,ch,h,l]
                        counter = counter + 1 
print(index_sorted.max())

NameError: name 'index_sorted' is not defined

In [33]:
frequency_h_n = 91
frequency_l_n = 26
d1 = np.memmap('../data/PAC_time_feature_5_30_10_100.mmap',dtype=float ,shape=(3, 25, 3, 63,frequency_h_n ,frequency_l_n,6),mode='r+')
print(d1.shape)
index_sorted = np.zeros(shape=(9*25*4*frequency_l_n*frequency_h_n,6))
counter = 0
for i in np.arange(3):
    for sub in np.arange(25):
        for j in np.arange(3):
            for ch in list_index:
                for h in np.arange(frequency_range_high.shape[0]):
                    for l in np.arange(frequency_range_low.shape[0]):
                        if np.any(d1[i,sub,j,ch,h,l,:]==0):
                            index_sorted[counter,:] = [i,sub,j,ch,h,l]
                        counter = counter + 1 
print(index_sorted.max())


(3, 25, 3, 63, 91, 26, 6)
0.0


In [10]:
frequency_h_n = 9
b = np.memmap('..\data\PAC_time_feature_8_30_25_100_v1_total_h_fre.mmap',dtype=float ,shape=(3, 25, 3, 63,frequency_h_n ,frequency_l_n,6),mode='r+')
print(b.shape)
index_sorted = np.zeros(shape=(9*25*4*9*23,6))
counter = 0
for i in np.arange(3):
    for sub in np.arange(25):
        for j in np.arange(3):
            for ch in list_index:
                for h in np.arange(frequency_range_high.shape[0]):
                    for l in np.arange(frequency_range_low.shape[0]):
                        if np.any(b[i,sub,j,ch,h,l,:]==0):
                            index_sorted[counter,:] = [i,sub,j,ch,h,l]
                        counter = counter + 1 
print(index_sorted.max())

frequency_h_n = 67
a = np.memmap('..\data\PAC_time_feature_8_30_25_100.mmap',dtype=float ,shape=(3, 25, 3, 63,frequency_h_n ,frequency_l_n,6),mode='r+')
print(a.shape)

index_sorted = np.zeros(shape=(9*25*4*67*23,6))
counter = 0
for i in np.arange(3):
    for sub in np.arange(25):
        for j in np.arange(3):
            for ch in list_index:
                for h in np.arange(67):
                    for l in np.arange(frequency_range_low.shape[0]):
                        if np.any(a[i,sub,j,ch,h,l,:]==0):
                            index_sorted[counter,:] = [i,sub,j,ch,h,l]
                        counter = counter + 1 

print(index_sorted.max()) 


(3, 25, 3, 63, 9, 23, 6)
0.0
(3, 25, 3, 63, 67, 23, 6)
0.0


In [11]:
total = np.zeros(shape=(3,25,3,63,76,23,6))
selected_index = frequency_range_high-25
counter_a = 0
counter_b = 0
for i in np.arange(76):
    if i in selected_index:
        total[:,:,:,:,i,:,:] = b[:,:,:,:,counter_b,:,:]
        counter_b = counter_b + 1
    else:
        total[:,:,:,:,i,:,:] = a[:,:,:,:,counter_a,:,:] 
        counter_a = counter_a + 1

In [12]:
index_sorted = np.zeros(shape=(9*25*4*76*23,6))
counter=0
for i in np.arange(3):
    for sub in np.arange(25):
        for j in np.arange(3):
            for ch in list_index:
                for h in np.arange(76):
                    for l in np.arange(frequency_range_low.shape[0]):
                        if np.any(total[i,sub,j,ch,h,l,:]==0):
                            index_sorted[counter,:] = [i,sub,j,ch,h,l]
                        counter = counter + 1 
print(index_sorted.max())

0.0


In [17]:
print(index_sorted.min)
np.all(total[:,:,:,[1,12,23,39],:,:,:]!=0)
total.shape

<built-in method min of numpy.ndarray object at 0x000001F3E0FB8C90>


(3, 25, 3, 63, 76, 23, 6)

In [18]:
c=np.memmap('..\data\PAC_time_feature_8_30_25_100_total.mmap',dtype=float ,shape=(3, 25, 3, 63,76 ,23,6),mode='w+')
# print(c.shape)
# print(total.shape)
# c[:,:,:,:,:,:,:]=total[:,:,:,:,:,:,:]



np.any(c!=total)

index_sorted = np.zeros(shape=(9*25*4*76*23,6))
counter=0
for i in np.arange(3):
    for sub in np.arange(25):
        for j in np.arange(3):
            for ch in list_index:
                for h in np.arange(76):
                    for l in np.arange(frequency_range_low.shape[0]):
                        if np.any(c[i,sub,j,ch,h,l,:]==0):
                            index_sorted[counter,:] = [i,sub,j,ch,h,l]
                        counter = counter + 1 
print(index_sorted.max())



(3, 25, 3, 63, 76, 23, 6)
(3, 25, 3, 63, 76, 23, 6)
0.0


In [20]:
np.all(c==total)


True

In [25]:
frequency_range_low.shape[0]

23

In [4]:
list_index

[1, 12, 23, 39]

In [10]:
d=np.memmap('../data/PAC_time_feature_8_30_25_100_total.mmap',dtype=float ,shape=(3, 25, 3, 63,76 ,23,6),mode='r+')
# print(np.any(d!=c))


window_size = 200 # in ms
down_sample_factor = 1
window_sample = int(window_size*(0.5/down_sample_factor))
frequency_h_n = 76
frequency_l_n = 23
frequency_range_high = np.linspace(25,100,frequency_h_n).astype(np.int64)
frequency_range_low = np.linspace(8,30,frequency_l_n).astype(np.int64)

frequency_range_high_1 = np.linspace(25,100,76).astype(np.int64)
frequency_range_high = np.setdiff1d(frequency_range_high_1,frequency_range_high)

index_sorted = np.zeros(shape=(9*25*4*76*23,6))
counter=0
for i in np.arange(3):
    for sub in np.arange(25):
        for j in np.arange(3):
            for ch in list_index:
                for h in np.arange(76):
                    for l in np.arange(frequency_range_low.shape[0]):
                        if np.any(d[i,sub,j,ch,h,l,:]==0):
                            index_sorted[counter,:] = [i,sub,j,ch,h,l]
                        counter = counter + 1 
print(index_sorted.max())


0.0


In [8]:
print(index_sorted.max())

0.0


In [3]:
a=np.memmap('..\data\PAC_time_feature_8_30_25_100.mmap',dtype=float ,shape=(3, 25, 3, 63,67 ,23,6),mode='r+')
print(a.shape)

# flag = np.zeros(shape=(9*25*4*67*23))

index_sorted = np.zeros(shape=(9*25*4*67*23,6))



window_size = 200 # in ms
down_sample_factor = 1
window_sample = int(window_size*(0.5/down_sample_factor))
frequency_h_n = 67
frequency_l_n = 23
frequency_range_high = np.linspace(25,100,frequency_h_n).astype(np.int64)
frequency_range_low = np.linspace(8,30,frequency_l_n).astype(np.int64)

print(frequency_range_high.shape[0])
counter = 0
for i in np.arange(3):
    for sub in np.arange(25):
        for j in np.arange(3):
            for ch in list_index:
                for h in np.arange(frequency_range_high.shape[0]):
                    for l in np.arange(frequency_range_low.shape[0]):
                        if np.any(a[i,sub,j,ch,h,l,:]==0):
                            index_sorted[counter,:] = [i,sub,j,ch,h,l]
                        counter = counter + 1     



(3, 25, 3, 63, 67, 23, 6)
67


In [5]:
selected_index = np.load('..\data\selected_index.npy')
selected_index.shape


(401196, 6)

In [17]:
b = np.sum(index_sorted,axis=1)
print(b.shape)
se_b = np.where(b!=0)
se_b[0].shape
counter = 2
selected_index = index_sorted[se_b[0],:]
print(selected_index.shape)
# np.save('selected_index.npy',selected_index)
# counter = np.random.randint(low=0,high=se_b[0].shape[0])
# print(counter)
# b = np.array([index_sort[se_b[0][counter],0],index_sort[se_b[0][counter],1],index_sort[se_b[0][counter],2],index_sort[se_b[0][counter],3],index_sort[se_b[0][counter],4],index_sort[se_b[0][counter],5]],dtype=int)

# print(a[b[0],b[1],b[2],b[3],b[4],b[5],:])

(1386900,)
(0, 6)


In [31]:
list_index

[39]

### Making index 

In [40]:
index_sorted = np.zeros(shape=(76*23,6))

window_size = 200 # in ms
down_sample_factor = 1
window_sample = int(window_size*(0.5/down_sample_factor))
frequency_h_n = 76
frequency_l_n = 23
frequency_range_high = np.linspace(25,100,frequency_h_n).astype(np.int64)
frequency_range_low = np.linspace(8,30,frequency_l_n).astype(np.int64)

# frequency_range_high_1 = np.linspace(25,100,76).astype(np.int64)
# frequency_range_high = np.setdiff1d(frequency_range_high_1,frequency_range_high)
counter=0
for i in np.arange(1):
    for sub in np.arange(1):
        for j in np.arange(1):
            for ch in list_index:
                for h in np.arange(frequency_range_high.shape[0]):
                    for l in np.arange(frequency_range_low.shape[0]):
                        index_sorted[counter,:] = [i,sub,j,ch,h,l]
                        counter = counter + 1

In [42]:
index_sorted.max()
print(index_sorted)


[[ 0.  0.  0. 39.  0.  0.]
 [ 0.  0.  0. 39.  0.  1.]
 [ 0.  0.  0. 39.  0.  2.]
 ...
 [ 0.  0.  0. 39. 75. 20.]
 [ 0.  0.  0. 39. 75. 21.]
 [ 0.  0.  0. 39. 75. 22.]]


In [44]:

window_size = 200 # in ms
down_sample_factor = 1
window_sample = int(window_size*(0.5/down_sample_factor))
frequency_h_n = 76
frequency_l_n = 23
frequency_range_high = np.linspace(25,100,frequency_h_n).astype(np.int64)
frequency_range_low = np.linspace(8,30,frequency_l_n).astype(np.int64)

PAC_feature_sub_on_electrodes = np.memmap('..\data\Test.mmap',dtype=float ,shape=(3, 25, 3, 63,frequency_h_n ,frequency_l_n,6),mode='r+')
# PAC_feature_sub_on_electrodes = np.zeros([3, 25, 3, 63,frequency_h_n ,frequency_l_n,6])
print(PAC_feature_sub_on_electrodes.flags)
def wrapper(indexes):
    i,sub,j,ch,h,l = int(indexes[0]),int(indexes[1]),int(indexes[2]),int(indexes[3]),int(indexes[4]),int(indexes[5])
    fre_l = frequency_range_low[l]
    fre_h = frequency_range_high[h] 
    for t in np.arange(6):
        PAC_feature_sub_on_electrodes[i,sub,j,ch,h,l,t] = pac.tfMVL(ERP_collection[i,sub,j,ch,t*window_sample:(t+1)*window_sample],[fre_h,fre_h],[fre_l,fre_l],500)


Parallel(n_jobs=12,verbose=10)(delayed(wrapper)(i) for i in index_sorted)

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False



[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.6s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    0.8s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    1.9s
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    2.9s
[Parallel(n_jobs=12)]: Done  37 tasks      | elapsed:    3.7s
[Parallel(n_jobs=12)]: Done  48 tasks      | elapsed:    4.5s
[Parallel(n_jobs=12)]: Done  61 tasks      | elapsed:    5.7s
[Parallel(n_jobs=12)]: Done  74 tasks      | elapsed:    6.9s
[Parallel(n_jobs=12)]: Done  89 tasks      | elapsed:    8.2s
[Parallel(n_jobs=12)]: Done 104 tasks      | elapsed:    9.5s
[Parallel(n_jobs=12)]: Done 121 tasks      | elapsed:   11.2s
[Parallel(n_jobs=12)]: Done 138 tasks      | elapsed:   12.7s
[Parallel(n_jobs=12)]: Done 157 tasks      | elapsed:   14.2s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:   15.8s
[Parallel(n_jobs=12)]: Done 197 tasks      | elapsed:  

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [ ]:
def wrapper(indexes):
    i,sub,j,ch,h,l = int(indexes[0]),int(indexes[1]),int(indexes[2]),int(indexes[3]),int(indexes[4]),int(indexes[5])
    fre_l = frequency_range_low[l]
    fre_h = frequency_range_high[h] 
    for t in np.arange(6):
        PAC_feature_sub_on_electrodes[i,sub,j,ch,h,l,t] = pac.tfMVL(ERP_collection[i,sub,j,ch,t*window_sample:(t+1)*window_sample],[fre_h,fre_h],[fre_l,fre_l],500)

for i in selected_index:
    wrapper(i)        

In [ ]:
np.where((frequency_range_high[1:-1]-frequency_range_high[0:-2])!=1)
i=1